# 1. 임의의 입력 생성하기

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

# 2D 텐서(timesteps, input_dim)
train_X = [ [0.1,4.2,1.5,1.1,2.8],
            [1.0,3.1,2.5,0.7,1.1],
            [0.3,2.1,1.5,2.1,0.1],
            [2.2,1.4,0.5,0.9,1.1] ]
print(np.shape(train_X))

(4, 5)


In [2]:
# 3D 텐서(batch_size, timesteps, input_dim)
train_X = [[ [0.1,4.2,1.5,1.1,2.8],
             [1.0,3.1,2.5,0.7,1.1],
             [0.3,2.1,1.5,2.1,0.1],
             [2.2,1.4,0.5,0.9,1.1] ]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


# 2. SimpleRNN 이해하기

In [3]:
# return_sequences=False: 마지막 시점 은닉 상태만 반환
rnn = SimpleRNN(3, return_sequences=False)
hidden_state = rnn(train_X)

print('hidden state: {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state: [[ 0.86486524 -0.95041156 -0.3999314 ]], shape: (1, 3)


In [4]:
# return_sequences=True: 모든 시점 은닉 상태를 반환
rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print('hidden states: {}, shape: {}'.format(hidden_states, hidden_states.shape))

hidden states: [[[ 0.3597447  -0.99311495  0.51452184]
  [ 0.99432474 -0.9975619   0.3476355 ]
  [ 0.8798514  -0.9492955  -0.5959873 ]
  [ 0.5912647  -0.7410295   0.5689865 ]]], shape: (1, 4, 3)


In [5]:
# return_state=True: return_sequences와 상관없이 마지막 시점 은닉 상태 반환
rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state: {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state: {}, shape: {}'.format(last_state, last_state.shape))

hidden state: [[ 0.9638611 -0.889439   0.9957025]], shape: (1, 3)
last hidden state: [[ 0.9638611 -0.889439   0.9957025]], shape: (1, 3)


In [6]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states: {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state: {}, shape: {}'.format(last_state, last_state.shape))

hidden states: [[[ 0.4814645   0.99550927  0.99875766]
  [ 0.3288639   0.9825352   0.8869835 ]
  [-0.8297914   0.86848813  0.8223589 ]
  [ 0.4004874   0.990376    0.41048357]]], shape: (1, 4, 3)
last hidden state: [[0.4004874  0.990376   0.41048357]], shape: (1, 3)


# 3. LSTM 이해하기

In [7]:
# return_state=True: 마지막 셀 상태까지 반환
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)

print('hidden state: {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state: {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state: {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state: [[0.1834961  0.7545921  0.02721529]], shape: (1, 3)
last hidden state: [[0.1834961  0.7545921  0.02721529]], shape: (1, 3)
last cell state: [[1.0024986  1.5968372  0.06269248]], shape: (1, 3)


In [8]:
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_state, last_cell_state = lstm(train_X)

print('hidden states: {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state: {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state: {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states: [[[0.5749347  0.38761458 0.00793081]
  [0.5119476  0.6050803  0.01128334]
  [0.5142331  0.61985    0.02179255]
  [0.53983325 0.29598406 0.00120676]]], shape: (1, 4, 3)
last hidden state: [[0.53983325 0.29598406 0.00120676]], shape: (1, 3)
last cell state: [[0.9415351  1.0041575  0.02211669]], shape: (1, 3)


# 4. Bidirectional LSTM 이해하기

In [9]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [10]:
# return_state=True: 마지막 시점 정방향 은닉/셀 상태와 역방향 은닉/셀 상태 4가지 반환
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True,
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states: {}, shape: {}'.format(hidden_states, hidden_states.shape))
# 정방향 LSTM 마지막 시점의 은닉 상태값
print('forward state: {}, shape: {}'.format(forward_h, forward_h.shape))
# 역방향 LSTM 첫번째 시점의 은닉 상태값
print('backward state: {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states: [[0.6303138 0.6303138 0.6303138 0.7038734 0.7038734 0.7038734]], shape: (1, 6)
forward state: [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state: [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


return_sequences=False: 정방향 LSTM 마지막 시점의 은닉 상태와, 역방향 LSTM 첫번째 시점의 은닉 상태가 연결되어 반환
![Bidirectional LSTM1](https://wikidocs.net/images/page/94748/bilstm3.PNG "Bidirectional LSTM1")

In [11]:
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True,
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states: {}, shape: {}'.format(hidden_states, hidden_states.shape))
# 정방향 LSTM 마지막 시점의 은닉 상태값
print('forward state: {}, shape: {}'.format(forward_h, forward_h.shape))
# 역방향 LSTM 첫번째 시점의 은닉 상태값
print('backward state: {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states: [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.55111325 0.55111325 0.55111325 0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.6303138  0.6303138  0.6303138  0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state: [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state: [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


return_sequences=True: 정방향 LSTM n번째 시점의 은닉 상태와, 역방향 LSTM n번째 시점의 은닉 상태가 연결되어 반환
![Bidirectional LSTM2](https://wikidocs.net/images/page/94748/bilstm1.PNG "Bidirectional LSTM2")